In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.losses import categorical_crossentropy
import skimage.transform
from skimage import img_as_ubyte
import numpy as np
from Datasets import Datasets
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.losses import categorical_crossentropy


class IdentityBlock(layers.Layer):
    def __init__(self, kernel_size, filters, **kwargs):
        super(IdentityBlock, self).__init__(**kwargs)
        self.kernel_size = kernel_size
        self.filters = filters

    def call(self, X):
        F1, F2, F3 = self.filters
        X_shortcut = X

        X = layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding="valid")(X)
        X = layers.BatchNormalization(axis=3)(X)
        X = layers.Activation("relu")(X)

        X = layers.Conv2D(filters=F2, kernel_size=(self.kernel_size, self.kernel_size), strides=(1, 1), padding="same")(X)
        X = layers.BatchNormalization(axis=3)(X)
        X = layers.Activation("relu")(X)

        X = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding="valid")(X)
        X = layers.BatchNormalization(axis=3)(X)

        X = layers.Add()([X, X_shortcut])
        X = layers.Activation("relu")(X)

        return X

    def compute_output_shape(self, input_shape):
        return input_shape

class ConvBlock(layers.Layer):
    def __init__(self, kernel_size, filters, strides = (2, 2), **kwargs):
        super(ConvBlock, self).__init__(**kwargs)
        self.kernel_size = kernel_size
        self.filters = filters

    def call(self, X):
        F1, F2, F3 = self.filters
        X_shortcut = X

        X = layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=strides, padding="valid")(X)
        X = layers.BatchNormalization(axis=3)(X)
        X = layers.Activation("relu")(X)

        X = layers.Conv2D(filters=F2, kernel_size=(self.kernel_size, self.kernel_size), strides=(1, 1), padding="same")(X)
        X = layers.BatchNormalization(axis=3)(X)
        X = layers.Activation("relu")(X)

        X = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding="valid")(X)
        X = layers.BatchNormalization(axis=3)(X)

        X_shortcut = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=strides, padding="valid")(X_shortcut)
        X_shortcut = layers.BatchNormalization(axis=3)(X_shortcut)

        X = layers.Add()([X, X_shortcut])
        X = layers.Activation("relu")(X)

        return X

    def compute_output_shape(self, input_shape):
        return input_shape


class ResNet(models.Sequential):
    name = "ResNet"
    input_shape = (64, 64, 3)

    def __init__(self, classes, optimizer):
        super().__init__()

        self.add(layers.ZeroPadding2D((3, 3)))
        self.add(layers.Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), input_shape=self.input_shape))
        self.add(layers.BatchNormalization(axis=3))
        self.add(layers.Activation('relu'))
        self.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

        # Stage 2
        self.add(ConvBlock(filters=(64, 64, 256), kernel_size=(3, 3), strides=(1, 1)))
        self.add(IdentityBlock(filters=(64, 64, 256), kernel_size=(64, 64, 256)))
        self.add(IdentityBlock(filters=(64, 64, 256), kernel_size=(64, 64, 256)))

        # output layer
        self.add(layers.Flatten())
        self.add(layers.Dense(classes, activation='softmax'))

        # Create model
        self.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=["accuracy"])

    def resize_images(self, images):
        tmp_images = []
        x, y, _ = self.input_shape
        for image in images:
            image = skimage.transform.resize(image, (x, y), mode='constant')
            image = img_as_ubyte(image)
            tmp_images.append(image)
        return np.array(tmp_images)





In [ ]:
X_train, Y_train, X_test, Y_test = Datasets.load_cifar10()

_, x_size, y_size, n_filters = X_train.shape
_, classes = Y_train.shape

print("got here")

model = ResNet(classes = classes, optimizer=keras.optimizers.SGD(learning_rate=0.01))
#model.compile(loss=categorical_crossentropy, optimizer=keras.optimizers.SGD(learning_rate=learning_rate), metrics=["accuracy"])

print("not here")

X_tr = model.resize_images(X_train[:200])
Y_tr = Y_train[:200]
X_vl = model.resize_images(X_test[:50])
Y_vl = Y_test[:50]

# Generate extra data
#datagen = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,fill_mode='nearest',horizontal_flip = True)
#datagen.fit(X_tr)

# Create history
batch_size = 128
print("and here")
history = model.fit(X_tr, Y_tr, batch_size=batch_size, validation_data=(X_vl, Y_vl), epochs=10)

got here
not here
and here
Epoch 1/10


TypeError: in user code:

    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:838 run_step  **
        outputs = model.tr

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=64d23a20-b5bc-41cc-930c-3bd7ede12b0c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>